In [1]:
import nltk
import csv
import ssl ## workaround for my specific authorization issue
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from rake_nltk import Rake


from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

from sklearn import svm

from gensim.models import Word2Vec
import time

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import classification_report

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df =pd.read_csv("out_0426.csv")

In [3]:
df_balanced = pd.concat([df[df["stars"]==5].sample(5000), df[df["stars"]==4].sample(5000), \
                         df[df["stars"]==3].sample(10000), df[df["stars"]==2].sample(5000), \
                         df[df["stars"]==1].sample(5000)], axis=0)

#Larger df
sample_size = 10000
# df_balanced = pd.concat([complete_df[df["stars"]==5].sample(sample_size), complete_df[df["stars"]==4].sample(sample_size), \
#                          complete_df[df["stars"]==3].sample(sample_size*2), complete_df[df["stars"]==2].sample(sample_size), \
#                          complete_df[df["stars"]==1].sample(sample_size)], axis=0)
df_balanced.head(10)

,user_id,stars,useful,text,tokenized_text,tokenized_text_string,compound,neg,neu,pos,star_revised
150239,G-9QboxmMfNLafH_X_HZvQ,5,1,Really friendly and honest service and quality...,"['realli', 'friendli', 'honest', 'servic', 'qu...",realli friendli honest servic qualiti work wel...,0.9201,0.045,0.660,0.296,1
149511,XIWKPSY9T57bJbn2lEF3ow,5,2,I have to admit that I had reservations about ...,"['admit', 'reserv', 'bar', 'pool', 'concern', ...",admit reserv bar pool concern diminish realiz ...,0.6124,0.000,0.737,0.263,1
189527,1q_YMI8NQjq93JKJzFpMDg,5,0,Put this restaurant on the top of your list fo...,"['put', 'restaur', 'top', 'list', 'north', 'en...",put restaur top list north end extrem delici m...,0.4939,0.083,0.736,0.181,1
131081,E_bvTNcbGT1k8fJT9Vegzw,5,0,Love this place! Excellent service and ambienc...,"['love', 'place', 'excel', 'servic', 'ambienc'...",love place excel servic ambienc best massag ev...,0.9413,0.000,0.468,0.532,1
141609,5zeOnCBHToQRKvdzBDDlSQ,5,0,Found this place on yelp thought we would give...,"['found', 'place', 'yelp', 'thought', 'would',...",found place yelp thought would give shot every...,0.3612,0.000,0.839,0.161,1
137738,FsY4mm3lLWykDB_f1GQBwg,5,0,I'm a fashionista and have quite a wardrobe (o...,"[""'m"", 'fashionista', 'quit', 'wardrob', 'ok',...",'m fashionista quit wardrob ok three closet ta...,0.7906,0.000,0.778,0.222,1
198008,-XS0hiiOrWyH_MJDBsq41Q,5,0,My friend had a birthday party here on Septemb...,"['friend', 'birthday', 'parti', 'septemb', '17...",friend birthday parti septemb 17th staff amaz ...,0.9524,0.000,0.499,0.501,1
83769,Cc-RQbnVWO6EM0mLyIweEg,5,0,Came to check out one of the most famous brunc...,"['came', 'check', 'one', 'famou', 'brunch', 'p...",came check one famou brunch place area colleag...,0.9442,0.000,0.728,0.272,1
29279,Pqa1rGz_gXDCbPVwvxykjw,5,0,We went in for sake social hour without a rese...,"['went', 'sake', 'social', 'hour', 'without', ...",went sake social hour without reserv parti 9. ...,0.9118,0.000,0.682,0.318,1
785,gjqVMjWhRrH8FuYbJSwfmA,5,0,This is fantastic! We had cold lobster rolls a...,"['fantast', 'cold', 'lobster', 'roll', 'shrimp...",fantast cold lobster roll shrimp burger server...,0.6808,0.043,0.804,0.153,1


In [4]:
len(df_balanced)

30000

In [5]:
df = df_balanced.reset_index()
df.head()

,index,user_id,stars,useful,text,tokenized_text,tokenized_text_string,compound,neg,neu,pos,star_revised
0,150239,G-9QboxmMfNLafH_X_HZvQ,5,1,Really friendly and honest service and quality...,"['realli', 'friendli', 'honest', 'servic', 'qu...",realli friendli honest servic qualiti work wel...,0.9201,0.045,0.660,0.296,1
1,149511,XIWKPSY9T57bJbn2lEF3ow,5,2,I have to admit that I had reservations about ...,"['admit', 'reserv', 'bar', 'pool', 'concern', ...",admit reserv bar pool concern diminish realiz ...,0.6124,0.000,0.737,0.263,1
2,189527,1q_YMI8NQjq93JKJzFpMDg,5,0,Put this restaurant on the top of your list fo...,"['put', 'restaur', 'top', 'list', 'north', 'en...",put restaur top list north end extrem delici m...,0.4939,0.083,0.736,0.181,1
3,131081,E_bvTNcbGT1k8fJT9Vegzw,5,0,Love this place! Excellent service and ambienc...,"['love', 'place', 'excel', 'servic', 'ambienc'...",love place excel servic ambienc best massag ev...,0.9413,0.000,0.468,0.532,1
4,141609,5zeOnCBHToQRKvdzBDDlSQ,5,0,Found this place on yelp thought we would give...,"['found', 'place', 'yelp', 'thought', 'would',...",found place yelp thought would give shot every...,0.3612,0.000,0.839,0.161,1


In [6]:
len(df)

30000

## TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
sentences = df["tokenized_text_string"]
# sentences = cleaned_df["text"]

vectorizer = TfidfVectorizer(norm = False, smooth_idf = False, min_df=0.04)
df["tfidf_vectors"] = vectorizer.fit_transform(sentences)
dd = vectorizer.fit_transform(sentences)
ddf =pd.DataFrame(dd.todense())
ddf

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,256,257
0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,5.874958,0.0,0.0,0.000000,6.351905,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,4.107036,0.000000,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,2.318262,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.0,0.0,0.0,4.095184,3.689228,0.0,0.0,0.0,0.0,0.000000,...,3.166017,0.0,2.937479,0.0,0.0,0.000000,0.000000,0.0,2.318262,0.0
29996,0.0,0.0,0.0,0.000000,3.689228,0.0,0.0,0.0,0.0,3.385604,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
29997,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,4.636524,0.0
29998,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0


## Hashing Vectorizer

In [77]:
from sklearn.feature_extraction.text import HashingVectorizer

sentences = df["tokenized_text_string"]

vectorizer = HashingVectorizer(norm = None, n_features = 500)
df["hashing_vectors"] = vectorizer.fit_transform(sentences)

dd = vectorizer.fit_transform(sentences)
ddh =pd.DataFrame(dd.todense())
ddh

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.0,0.0,0.0,0.0,3.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29996,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,-1.0
29997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
29998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Word2Vec

In [61]:
from gensim.models import Word2Vec
import time
# Skip-gram model (sg = 1)
# size = 1000
window = 3
min_count = 3
workers = 3
sg = 1

word2vec_model_file = 'word2vec_' + str(size) + '.model'
start_time = time.time()
stemmed_tokens = pd.Series(df['tokenized_text']).values
# Train the Word2Vec Model
# w2v_model = Word2Vec(stemmed_tokens, min_count = min_count, size = size, workers = workers, window = window, sg = sg)
w2v_model = Word2Vec(stemmed_tokens, min_count = 3, vector_size=500)
print("Time taken to train word2vec model: " + str(time.time() - start_time))
w2v_model.save(word2vec_model_file)

Time taken to train word2vec model: 22.433595180511475


In [62]:
# Load the model from the model file
sg_w2v_model = Word2Vec.load(word2vec_model_file)
# Unique ID of the word
print("Index of the word 'food':")
print(sg_w2v_model.wv.key_to_index["a"])
# Total number of the words 
print(len(sg_w2v_model.wv))
# Print the size of the word2vec vector for one word
print("Length of the vector generated for a word")
print(len(sg_w2v_model.wv.get_vector("a", norm=True)))
# Get the mean for the vectors for an example review
print("Print the length after taking average of all word vectors in a sentence:")
# print(np.mean([sg_w2v_model[token] for token in df['tokenized_text'][0]], axis=0))
print(np.mean([sg_w2v_model.wv.get_vector(token, norm=True) for token in df['tokenized_text'][11243]], axis=0))

Index of the word 'food':
4
168
Length of the vector generated for a word
500
Print the length after taking average of all word vectors in a sentence:
[ 4.61328309e-03  2.02159472e-02  3.84073704e-02 -9.45460517e-03
  6.94192061e-03 -2.30161846e-03 -4.18951176e-02 -1.54208373e-02
  5.31395152e-02 -2.48199515e-02  5.09252623e-02 -3.73319344e-04
 -1.16964178e-02  4.73887436e-02  3.18159699e-03 -2.62003709e-02
 -7.20308870e-02 -6.14148825e-02  1.74069330e-02  5.48167564e-02
 -1.53883100e-02 -1.84926279e-02  4.03459854e-02 -1.00154812e-02
  2.38929372e-02 -2.77522448e-02  2.46921182e-02  1.00365020e-02
 -8.50715339e-02 -3.64152826e-02  2.68265270e-02 -1.12795853e-03
 -1.43646784e-02  1.81232784e-02 -1.26236174e-02 -1.28110033e-02
  1.71790831e-02 -1.38360737e-02 -6.77506700e-02  3.74902016e-03
 -2.49507483e-02  5.01490757e-03 -4.83370423e-02 -2.48337584e-03
 -8.52391508e-07  1.63933297e-03 -1.20047517e-02 -3.11846682e-03
 -1.42088830e-02  2.81406362e-02  5.39613292e-02  1.93400066e-02
 -1.

In [63]:
# def vecc(stem):
#     return np.mean([sg_w2v_model.wv.get_vector(token, norm=True) for token in stem], axis=0)
# ddw = df["tokenized_text"].apply(vecc)
def vecc(stem):
    stem_list = []
    for token in stem:
        if token not in sg_w2v_model.wv:
            continue
        else:
            stem_list.append(token)
    return np.mean([sg_w2v_model.wv.get_vector(token, norm=True) for token in stem_list], axis=0)

In [64]:
ddw = df["tokenized_text"].apply(vecc)
ddw.head()

0    [0.0046236985, 0.021110188, 0.036307074, -0.00...
1    [0.0062831948, 0.021519918, 0.038008504, -0.00...
2    [0.0009752877, 0.02158588, 0.03583831, -0.0087...
3    [0.006042352, 0.016734581, 0.034428276, -0.008...
4    [0.0019621283, 0.022269433, 0.03513606, -0.009...
Name: tokenized_text, dtype: object

In [65]:
ddw = ddw.tolist()

In [66]:
ddw = pd.DataFrame(ddw)
len(ddw)

30000

## Basic ML DF

In [78]:
ml = df[["stars","compound","neg","neu","pos","star_revised"]]
len(ml)

30000

In [79]:
ml.head()

,stars,compound,neg,neu,pos,star_revised
0,5,0.9201,0.045,0.660,0.296,1
1,5,0.6124,0.000,0.737,0.263,1
2,5,0.4939,0.083,0.736,0.181,1
3,5,0.9413,0.000,0.468,0.532,1
4,5,0.3612,0.000,0.839,0.161,1


## Complete DF

In [80]:
complete_df = pd.concat([ml, ddf, ddh, ddw], axis=1)
len(complete_df)

30000

In [81]:
complete_df.head(10)

,stars,compound,neg,neu,pos,star_revised,0,1,2,3,...,490,491,492,493,494,495,496,497,498,499
0,5,0.9201,0.045,0.660,0.296,1,0.0,0.0,0.0,0.0,...,0.025479,-0.001252,-0.000337,0.079724,0.035832,0.058776,-0.015807,0.034695,-0.035199,-0.018140
1,5,0.6124,0.000,0.737,0.263,1,0.0,0.0,0.0,0.0,...,0.020964,0.002355,0.005168,0.086876,0.038500,0.057599,-0.016247,0.033774,-0.036672,-0.012167
2,5,0.4939,0.083,0.736,0.181,1,0.0,0.0,0.0,0.0,...,0.026155,-0.001190,0.000217,0.083389,0.039452,0.059277,-0.020746,0.043339,-0.033814,-0.013813
3,5,0.9413,0.000,0.468,0.532,1,0.0,0.0,0.0,0.0,...,0.022619,0.000910,0.004690,0.074881,0.035899,0.056349,-0.017943,0.032241,-0.034965,-0.014337
4,5,0.3612,0.000,0.839,0.161,1,0.0,0.0,0.0,0.0,...,0.026120,-0.000600,-0.002621,0.072744,0.035144,0.058046,-0.014720,0.035711,-0.033902,-0.018021
5,5,0.7906,0.000,0.778,0.222,1,0.0,0.0,0.0,0.0,...,0.025158,-0.001857,0.000929,0.077461,0.034615,0.056522,-0.017476,0.035500,-0.031531,-0.015296
6,5,0.9524,0.000,0.499,0.501,1,0.0,0.0,0.0,0.0,...,0.026148,-0.001914,-0.000846,0.073509,0.034487,0.058203,-0.016634,0.035116,-0.034234,-0.014882
7,5,0.9442,0.000,0.728,0.272,1,0.0,0.0,0.0,0.0,...,0.025787,-0.002026,-0.000729,0.080968,0.037850,0.057780,-0.013949,0.035635,-0.036396,-0.013551
8,5,0.9118,0.000,0.682,0.318,1,0.0,0.0,0.0,0.0,...,0.029050,-0.001544,0.004199,0.078832,0.038133,0.060599,-0.016380,0.037354,-0.039794,-0.014178
9,5,0.6808,0.043,0.804,0.153,1,0.0,0.0,0.0,0.0,...,0.027486,-0.001794,-0.000298,0.079068,0.031730,0.058946,-0.012998,0.036071,-0.031428,-0.020105


In [82]:
# complete_df = df_balanced
len(complete_df)

30000

In [83]:
# compression_opts = dict(method='zip', archive_name='out.csv')
# complete_df.to_csv('complete.zip', index=False, compression=compression_opts)

In [84]:
complete_df.iloc[:,6:]

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.025479,-0.001252,-0.000337,0.079724,0.035832,0.058776,-0.015807,0.034695,-0.035199,-0.018140
1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.020964,0.002355,0.005168,0.086876,0.038500,0.057599,-0.016247,0.033774,-0.036672,-0.012167
2,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.026155,-0.001190,0.000217,0.083389,0.039452,0.059277,-0.020746,0.043339,-0.033814,-0.013813
3,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.022619,0.000910,0.004690,0.074881,0.035899,0.056349,-0.017943,0.032241,-0.034965,-0.014337
4,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.026120,-0.000600,-0.002621,0.072744,0.035144,0.058046,-0.014720,0.035711,-0.033902,-0.018021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.0,0.0,0.0,4.095184,3.689228,0.0,0.0,0.0,0.0,0.000000,...,0.027295,-0.002094,0.001221,0.077256,0.037896,0.057884,-0.018606,0.039836,-0.033784,-0.013651
29996,0.0,0.0,0.0,0.000000,3.689228,0.0,0.0,0.0,0.0,3.385604,...,0.026851,-0.001963,0.002633,0.079377,0.037052,0.060057,-0.015880,0.038350,-0.036107,-0.014008
29997,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.030797,0.000698,0.004981,0.076533,0.037650,0.057861,-0.021699,0.040370,-0.044048,-0.012995
29998,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.029006,-0.005937,0.003085,0.081749,0.032679,0.060707,-0.015598,0.037641,-0.028765,-0.014779


## 优先测试SVM

## Polarity_Score

In [91]:
X = complete_df[["compound","neg","neu","pos"]]
# X = X.head(10000)
y = complete_df["star_revised"]
# y = y.head(10000)

X_train, X_final_test, y_train, y_final_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, random_state=123)

scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [42]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train_scaled, y_train)

y_pred = clf.predict(X_test_scaled)
acc_test = (accuracy_score(y_pred,y_test))
y_pred = clf.predict(X_train_scaled)
acc_train = (accuracy_score(y_pred,y_train))
# print('kernel = rbf')
print('train accuracy: {}'.format(acc_train))
print('test accuracy: {}'.format(acc_test))

train accuracy: 0.5218518518518519
test accuracy: 0.519074074074074


In [43]:
y_test_pred = clf.predict(scaler.transform(X_final_test))
acc_test = (accuracy_score(y_test_pred,y_final_test))
acc_test

0.529

In [44]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()
rmfr.fit(X_train_scaled,y_train)
predrmfr = rmfr.predict(X_test_scaled)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:",classification_report(y_test,predrmfr))

Confusion Matrix for Random Forest Classifier:
[[1015  511  282]
 [ 586  606  569]
 [ 359  549  923]]
Score: 47.11
Classification Report:               precision    recall  f1-score   support

          -1       0.52      0.56      0.54      1808
           0       0.36      0.34      0.35      1761
           1       0.52      0.50      0.51      1831

    accuracy                           0.47      5400
   macro avg       0.47      0.47      0.47      5400
weighted avg       0.47      0.47      0.47      5400



## 只有TF-IDF

In [92]:
X = complete_df.iloc[:,6:262]
# X = X.sample(10000)
y = complete_df["star_revised"]
# y = y.sample(10000)

X_train, X_final_test, y_train, y_final_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, random_state=123)

scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [46]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()
rmfr.fit(X_train_scaled,y_train)
predrmfr = rmfr.predict(X_test_scaled)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:",classification_report(y_test,predrmfr))

Confusion Matrix for Random Forest Classifier:
[[1221  410  177]
 [ 396 1006  359]
 [ 211  417 1203]]
Score: 63.52
Classification Report:               precision    recall  f1-score   support

          -1       0.67      0.68      0.67      1808
           0       0.55      0.57      0.56      1761
           1       0.69      0.66      0.67      1831

    accuracy                           0.64      5400
   macro avg       0.64      0.63      0.64      5400
weighted avg       0.64      0.64      0.64      5400



In [38]:
from sklearn import svm
clf2 = svm.SVC()
clf2.fit(X_train_scaled, y_train)

y_pred = clf2.predict(X_test_scaled)
acc_test = (accuracy_score(y_pred,y_test))
y_pred = clf2.predict(X_train_scaled)
acc_train = (accuracy_score(y_pred,y_train))
# print('kernel = rbf')
print('train accuracy: {}'.format(acc_train))
print('test accuracy: {}'.format(acc_test))

train accuracy: 0.8063888888888889
test accuracy: 0.6468518518518519


In [41]:
y_test_pred = clf2.predict(scaler.transform(X_final_test))
acc_test = (accuracy_score(y_test_pred,y_final_test))
acc_test

0.6453333333333333

## 只有 Hashing

In [85]:
X = complete_df.iloc[:,264:764]
# X = X.sample(10000)
y = complete_df["star_revised"]
# y = y.sample(10000)

X_train, X_final_test, y_train, y_final_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, random_state=123)

scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [86]:
X

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.0,0.0,0.0,0.0,3.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29996,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,-1.0
29997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
29998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()
rmfr.fit(X_train_scaled,y_train)
predrmfr = rmfr.predict(X_test_scaled)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:",classification_report(y_test,predrmfr))

Confusion Matrix for Random Forest Classifier:
[[1245  385  178]
 [ 409  967  385]
 [ 211  449 1171]]
Score: 62.65
Classification Report:               precision    recall  f1-score   support

          -1       0.67      0.69      0.68      1808
           0       0.54      0.55      0.54      1761
           1       0.68      0.64      0.66      1831

    accuracy                           0.63      5400
   macro avg       0.63      0.63      0.63      5400
weighted avg       0.63      0.63      0.63      5400



In [88]:
from sklearn import svm
clf3 = svm.SVC()
clf3.fit(X_train_scaled, y_train)

y_pred = clf3.predict(X_test_scaled)
acc_test = (accuracy_score(y_pred,y_test))
y_pred = clf3.predict(X_train_scaled)
acc_train = (accuracy_score(y_pred,y_train))
# print('kernel = rbf')
print('train accuracy: {}'.format(acc_train))
print('test accuracy: {}'.format(acc_test))

train accuracy: 0.6432407407407408
test accuracy: 0.6316666666666667


In [89]:
y_test_pred = clf3.predict(scaler.transform(X_final_test))
acc_test = (accuracy_score(y_test_pred,y_final_test))
acc_test

0.6313333333333333

## 只有Word2Vec

In [74]:
X = complete_df.iloc[:,464:]
# X = X.sample(10000)
y = complete_df["star_revised"]
# y = y.sample(10000)

X_train, X_final_test, y_train, y_final_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, random_state=123)

scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [75]:
X

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.004624,0.021110,0.036307,-0.008094,0.005913,-0.002136,-0.042194,-0.014009,0.055491,-0.027915,...,0.025479,-0.001252,-0.000337,0.079724,0.035832,0.058776,-0.015807,0.034695,-0.035199,-0.018140
1,0.006283,0.021520,0.038009,-0.006134,0.007763,-0.001434,-0.034650,-0.012647,0.055716,-0.019628,...,0.020964,0.002355,0.005168,0.086876,0.038500,0.057599,-0.016247,0.033774,-0.036672,-0.012167
2,0.000975,0.021586,0.035838,-0.008728,0.013523,-0.000768,-0.044378,-0.020383,0.058289,-0.028483,...,0.026155,-0.001190,0.000217,0.083389,0.039452,0.059277,-0.020746,0.043339,-0.033814,-0.013813
3,0.006042,0.016735,0.034428,-0.008510,0.011022,0.005947,-0.032423,-0.016576,0.053298,-0.021341,...,0.022619,0.000910,0.004690,0.074881,0.035899,0.056349,-0.017943,0.032241,-0.034965,-0.014337
4,0.001962,0.022269,0.035136,-0.009120,0.011726,-0.003346,-0.045192,-0.016486,0.054826,-0.023152,...,0.026120,-0.000600,-0.002621,0.072744,0.035144,0.058046,-0.014720,0.035711,-0.033902,-0.018021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,-0.000601,0.017865,0.034523,-0.011333,0.010673,-0.000150,-0.041918,-0.020382,0.054674,-0.028080,...,0.027295,-0.002094,0.001221,0.077256,0.037896,0.057884,-0.018606,0.039836,-0.033784,-0.013651
29996,-0.001038,0.017846,0.033512,-0.012071,0.009722,-0.002401,-0.041643,-0.020298,0.055467,-0.025046,...,0.026851,-0.001963,0.002633,0.079377,0.037052,0.060057,-0.015880,0.038350,-0.036107,-0.014008
29997,-0.000631,0.015208,0.034255,-0.013178,0.010434,0.000639,-0.041394,-0.020856,0.050060,-0.024663,...,0.030797,0.000698,0.004981,0.076533,0.037650,0.057861,-0.021699,0.040370,-0.044048,-0.012995
29998,-0.002040,0.016468,0.029288,-0.017841,0.011196,-0.001267,-0.036147,-0.026944,0.055485,-0.037703,...,0.029006,-0.005937,0.003085,0.081749,0.032679,0.060707,-0.015598,0.037641,-0.028765,-0.014779


In [76]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()
rmfr.fit(X_train_scaled,y_train)
predrmfr = rmfr.predict(X_test_scaled)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:",classification_report(y_test,predrmfr))

Confusion Matrix for Random Forest Classifier:
[[922 429 457]
 [523 687 551]
 [406 515 910]]
Score: 46.65
Classification Report:               precision    recall  f1-score   support

          -1       0.50      0.51      0.50      1808
           0       0.42      0.39      0.41      1761
           1       0.47      0.50      0.49      1831

    accuracy                           0.47      5400
   macro avg       0.46      0.47      0.46      5400
weighted avg       0.47      0.47      0.47      5400



In [53]:
# Support Vector Machine
from sklearn.svm import SVC
svm = SVC(random_state=101)
svm.fit(X_train_scaled,y_train)
predsvm = svm.predict(X_test_scaled)
print("Confusion Matrix for Support Vector Machines:")
print(confusion_matrix(y_test,predsvm))
print("Score:",round(accuracy_score(y_test,predsvm)*100,2))
print("Classification Report:",classification_report(y_test,predsvm))

Confusion Matrix for Support Vector Machines:
[[989 434 385]
 [570 658 533]
 [448 475 908]]
Score: 47.31
Classification Report:               precision    recall  f1-score   support

          -1       0.49      0.55      0.52      1808
           0       0.42      0.37      0.40      1761
           1       0.50      0.50      0.50      1831

    accuracy                           0.47      5400
   macro avg       0.47      0.47      0.47      5400
weighted avg       0.47      0.47      0.47      5400



In [94]:
y_pred = svm.predict(X_test_scaled)
acc_test = (accuracy_score(y_pred,y_test))
y_pred = svm.predict(X_train_scaled)
acc_train = (accuracy_score(y_pred,y_train))
# print('kernel = rbf')
print('train accuracy: {}'.format(acc_train))
print('test accuracy: {}'.format(acc_test))

train accuracy: 0.4764814814814815
test accuracy: 0.47314814814814815


In [95]:
y_test_pred = svm.predict(scaler.transform(X_final_test))
acc_test = (accuracy_score(y_test_pred,y_final_test))
acc_test

0.4723333333333333

## All 

In [103]:
X = complete_df.drop(["stars","star_revised"], axis = 1)
# X = X.sample(10000)
y = complete_df["star_revised"]
# y = y.sample(10000)

X_train, X_final_test, y_train, y_final_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=123)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, random_state=123)

scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [69]:
X

,compound,neg,neu,pos,0,1,2,3,4,5,...,90,91,92,93,94,95,96,97,98,99
0,0.8402,0.048,0.762,0.190,0.000000,0.0,0.0,0.0,0.000000,2.745258,...,0.105875,-0.033451,-0.026138,0.024679,-0.037601,-0.061000,-0.166048,0.052458,-0.118615,0.072526
1,0.9118,0.000,0.792,0.208,3.717091,0.0,0.0,0.0,0.000000,0.000000,...,0.105677,-0.039176,-0.027938,0.023725,-0.036364,-0.061635,-0.172103,0.051059,-0.119155,0.071409
2,0.8807,0.000,0.469,0.531,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.117437,-0.028717,-0.022030,0.019610,-0.039178,-0.067642,-0.170202,0.057510,-0.118782,0.075258
3,0.7676,0.149,0.567,0.284,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.115316,-0.032853,-0.025235,0.023174,-0.034649,-0.067232,-0.162817,0.054005,-0.118994,0.073094
4,0.7964,0.089,0.584,0.327,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.098804,-0.030305,-0.021620,0.028816,-0.040456,-0.065448,-0.178661,0.054761,-0.127731,0.077840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.2263,0.079,0.826,0.094,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.111577,-0.031301,-0.029552,0.023044,-0.041849,-0.070420,-0.166805,0.056540,-0.116551,0.068233
29996,0.0000,0.000,1.000,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.082966,-0.024784,-0.015498,0.016486,-0.008023,-0.070252,-0.156827,0.073164,-0.102425,0.054823
29997,-0.6369,0.129,0.871,0.000,0.000000,0.0,0.0,0.0,0.000000,2.745258,...,0.110631,-0.036877,-0.030233,0.028657,-0.034333,-0.075313,-0.178338,0.054175,-0.123067,0.073069
29998,0.2235,0.000,0.946,0.054,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.109490,-0.034383,-0.028117,0.018882,-0.033832,-0.072291,-0.167768,0.059603,-0.119954,0.077736


In [104]:
# Support Vector Machine
from sklearn.svm import SVC
svm = SVC(random_state=101)
svm.fit(X_train_scaled,y_train)
predsvm = svm.predict(X_test_scaled)
print("Confusion Matrix for Support Vector Machines:")
print(confusion_matrix(y_test,predsvm))
print("Score:",round(accuracy_score(y_test,predsvm)*100,2))
print("Classification Report:",classification_report(y_test,predsvm))

Confusion Matrix for Support Vector Machines:
[[1237  416  155]
 [ 356  985  420]
 [ 154  324 1353]]
Score: 66.2
Classification Report:               precision    recall  f1-score   support

          -1       0.71      0.68      0.70      1808
           0       0.57      0.56      0.57      1761
           1       0.70      0.74      0.72      1831

    accuracy                           0.66      5400
   macro avg       0.66      0.66      0.66      5400
weighted avg       0.66      0.66      0.66      5400



In [105]:
y_pred = svm.predict(X_test_scaled)
acc_test = (accuracy_score(y_pred,y_test))
y_pred = svm.predict(X_train_scaled)
acc_train = (accuracy_score(y_pred,y_train))
# print('kernel = rbf')
print('train accuracy: {}'.format(acc_train))
print('test accuracy: {}'.format(acc_test))

train accuracy: 0.6712037037037037
test accuracy: 0.6620370370370371


In [106]:
y_test_pred = svm.predict(scaler.transform(X_final_test))
acc_test = (accuracy_score(y_test_pred,y_final_test))
acc_test

0.6616666666666666

## 看SVM结果如何了

In [88]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier(max_depth=10)
rmfr.fit(X_train_scaled,y_train)
predrmfr = rmfr.predict(X_test_scaled)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrmfr))
print("Score:",round(accuracy_score(y_test,predrmfr)*100,2))
print("Classification Report:",classification_report(y_test,predrmfr))

Confusion Matrix for Random Forest Classifier:
[[1246  360  202]
 [ 482  792  487]
 [ 239  376 1216]]
Score: 60.26
Classification Report:               precision    recall  f1-score   support

          -1       0.63      0.69      0.66      1808
           0       0.52      0.45      0.48      1761
           1       0.64      0.66      0.65      1831

    accuracy                           0.60      5400
   macro avg       0.60      0.60      0.60      5400
weighted avg       0.60      0.60      0.60      5400



In [100]:
tr = []
tes= []
fin=[]

for i in [2,4,6,8,10,12,14,16,18,20]:
    rmfr = RandomForestClassifier(max_depth=i)
    rmfr.fit(X_train_scaled,y_train)

    y_pred = rmfr.predict(X_train_scaled)
    acc_train = (accuracy_score(y_pred,y_train))

    y_pred = rmfr.predict(X_test_scaled)
    acc_test = (accuracy_score(y_pred,y_test))
    
    tr.append(acc_train)
    tes.append(acc_test)

    print('train accuracy: {}'.format(acc_train))
    print('test accuracy: {}'.format(acc_test))
    
    y_test_final = rmfr.predict(scaler.transform(X_final_test))
    acc_test = (accuracy_score(y_test_final,y_final_test))
    fin.append(acc_test)

train accuracy: 0.4387962962962963
test accuracy: 0.44037037037037036
train accuracy: 0.4637962962962963
test accuracy: 0.4559259259259259
train accuracy: 0.5099537037037037
test accuracy: 0.4664814814814815
train accuracy: 0.6019907407407408
test accuracy: 0.4722222222222222
train accuracy: 0.7526851851851852
test accuracy: 0.4674074074074074
train accuracy: 0.8868981481481482
test accuracy: 0.46703703703703703
train accuracy: 0.9660648148148148
test accuracy: 0.4635185185185185
train accuracy: 0.9910185185185185
test accuracy: 0.4614814814814815
train accuracy: 0.9990740740740741
test accuracy: 0.4653703703703704
train accuracy: 0.9997685185185186
test accuracy: 0.4737037037037037
